### Setup

In [ ]:
!pip install geopandas # Tables with spatial queries

In [ ]:
# Recipe for installing Cartopy: https://github.com/googlecolab/colabtools/issues/85
#!apt-get install libproj-dev proj-data proj-bin
#!apt-get install libgeos-dev
#!pip install cython
!pip install cartopy # Maps

In [ ]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd

from matplotlib import pyplot as plt

In [ ]:
import cartopy.io.img_tiles as cimgt

In [ ]:
#import cartopy

In [ ]:
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)










### Import tables

In [ ]:
import os.path as osp

In [ ]:
DATA_PATH = osp.join(os.environ['DATA_PATH'], 'cciw/Data')

In [ ]:
#DATA_PATH = r'/content/drive/My Drive/Data'
analysis_path = os.path.join(DATA_PATH, 'Tables', 'Analysis.csv')
dive_path = os.path.join(DATA_PATH, 'Tables', 'Dives.csv')
site_path = os.path.join(DATA_PATH, 'Tables', 'Sites.csv')
merged_path = os.path.join(DATA_PATH, 'Tables', 'MergedTable.csv')
simplified_path = os.path.join(DATA_PATH, 'Tables', 'SimplifiedImagingAnalysis.csv')

In [ ]:
analysis_df = pd.read_csv(analysis_path, index_col=0,
                          dtype={'Count':float})
dive_df = pd.read_csv(dive_path, index_col=0, parse_dates=['Date'])
site_df = pd.read_csv(site_path, index_col=0)
merged_df = pd.read_csv(merged_path, index_col=0)
simplified_df = pd.read_csv(simplified_path, index_col=0)

### Map diving sites on map

In [ ]:
site_df.set_index('PSN')

In [ ]:
cimgt.Stamen?

In [ ]:
stamen_terrain = cimgt.Stamen(style='terrain-background')
 # Load basemap online

In [ ]:
# Convert DataFrame to GeoDataFrame with WGS-1984 (GPS latitude/longitude) coordinate reference system 
site_gdf = gpd.GeoDataFrame(site_df,
                            crs={'init': 'epsg:4326'},
                            geometry=gpd.points_from_xy(site_df.Longitude, site_df.Latitude))

In [ ]:
# Reproject site coordinates to basemap crs
site_gdf = site_gdf.to_crs(stamen_terrain.crs.proj4_params)

In [ ]:
# Map sites on basemap
#fig = plt.figure(figsize=(15, 15))
fig = plt.figure(figsize=(12, 7))
ax = plt.axes(projection=stamen_terrain.crs)
ax.set_extent([-81.8, -77.2, 42.2, 44.2])
ax.add_image(stamen_terrain, 8)
#site_gdf.plot(ax=ax, markersize=50, marker='o', facecolor='none', edgecolor='k')
site_gdf.plot(ax=ax, markersize=50, facecolor='k')
plt.tight_layout()
#fig.savefig('dive-sites.eps', format='eps')
fig.savefig('dive-sites.jpg', format='jpg')

In [ ]:
#site_gdf.plot?

### Dive table exploration

In [ ]:
dive_df

In [ ]:
# All boat cruise campaigns
all_cruises = dive_df['Cruise #'].unique()
all_cruises

In [ ]:
print(f"Total number of boat cruises: {len(all_cruises)}")

In [ ]:
# Count number of dives for each day
dive_df_by_date = dive_df.groupby(by='Date')['PSN'].count()
dive_df_by_date

In [ ]:
print(f"Total number of days of diving: {len(dive_df_by_date)}")

In [ ]:
print(f"Date range for diving: {dive_df_by_date.index.min().strftime('%Y-%b-%d')} \
to {dive_df_by_date.index.max().strftime('%Y-%b-%d')}")

In [ ]:
# Frequency for number of dives per day
dive_df_by_date.hist(bins=[0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5,10.5])

In [ ]:
# Number of dives for each year-month
dive_df['Year-Month'] = dive_df['Date'].dt.strftime('%Y-%m')
dive_df_by_year_month = dive_df.groupby(by='Year-Month').count()['PSN']
dive_df_by_year_month.index = [datetime.strptime(year_month, '%Y-%m') for year_month in dive_df_by_year_month.index]
dive_df_by_year_month

In [ ]:
def line_format(label): # https://stackoverflow.com/questions/30133280/pandas-bar-plot-changes-date-format
    """
    Convert time label to the format of pandas line plot
    """
    month = label.month_name()[:3]
    if month == 'Jul':
        month += f'\n{label.year}'
    elif month in ['Apr','May','Jun']:
        month += f'\n>'
    elif month in ['Aug','Sep','Oct']:
        month += f'\n<'
    return month

In [ ]:
# Note that we specify rot here
ax = dive_df_by_year_month.plot(kind='bar', figsize=(12, 8), color='#2ecc71', rot=0)
ax.set_xticklabels(map(lambda x: line_format(x), dive_df_by_year_month.index))

In [ ]:
# Number of dives for each year
dive_df['Year'] = dive_df['Date'].dt.year
dive_df_by_year = dive_df.groupby(by='Year').count()['PSN']
#dive_df_by_year.index = [datetime.strptime(year, '%Y-%m') for year_month in dive_df_by_year_month.index]
dive_df_by_year

In [ ]:
dive_df_by_year.plot.bar()

In [ ]:
# Depth in meters of the lake bed at the diving site
dive_df['Depth (m)'].hist()

In [ ]:
# Overall percentage mussels coverage for the whole extent of the diving site (as estimated visually by diver)
dive_df['Overall Coverage'].hist()
# Note: this should not be used to estimate percentage coverage within a quadrat

In [ ]:
# Estimated substrate proportion of the lake bed
dive_substrate_df = dive_df[['Silt (%)', 'Clay (%)', 'Sand (%)', 'Gravel (%)', 
                             'Cobble (%)', 'Rock (%)', 'Bedrock (%)', 
                             'Boulders (%)', 'Shale (%)']]
dive_substrate_df

In [ ]:
fig = plt.figure(figsize=(15,15))
ax = fig.gca()
dive_substrate_df.hist(ax=ax)

In [ ]:
dive_df['Underlying Substrate Type'].unique()

In [ ]:
dive_df['Underlying Substrate Depth (cm)'].unique()

### Analysis Table Exploration

In [ ]:
analysis_df = analysis_df.set_index('Analysis Index')
analysis_df

In [ ]:
analysis_df[['Live Coverage', 'Empty Coverage']].hist(figsize=(12,4))

In [ ]:
# Frequency of mussels count within a quadrat
plt.hist(analysis_df['Count'].values, bins=[0]+list(np.logspace(np.log10(1),np.log10(10000), 20)))
plt.gca().set_xscale("log")
plt.xlabel('Count (#)')

In [ ]:
# Frequency of mussels biomass in grams within a quadrat
plt.hist(analysis_df['Biomass'].values, bins=[0]+list(np.logspace(np.log10(1),np.log10(10000), 20)))
plt.gca().set_xscale("log")
plt.xlabel('Biomass (g)')

In [ ]:
analysis_df[['16mm', '14mm', '12.5mm', '10mm', '8mm', '6.3mm', '4mm', '2mm']].hist(figsize=(15,15))

### Count number of rows for each analysis type

In [ ]:
n_alive = len(analysis_df['Live Coverage'].dropna())
print(f"Number of % live coverage data: {n_alive}")

n_empty = len(analysis_df['Empty Coverage'].dropna())
print(f"Number of % empty coverage data: {n_empty}")

n_biomass = len(analysis_df['Biomass'].dropna())
print(f"Number of biomass data: {n_biomass}")

n_count = len(analysis_df['Count'].dropna())
print(f"Number of count data: {n_count}")

n_bysize = len(analysis_df['10mm'].dropna())
print(f"Number of sorted mussels size data: {n_bysize}")

In [ ]:
merged_df.columns

In [ ]:
n_alive = len(merged_df['Live Coverage'].dropna())
print(f"Number of % live coverage data associated with image/video/still: {n_alive}")

n_empty = len(merged_df['Empty Coverage'].dropna())
print(f"Number of % empty coverage data associated with image/video/still: {n_empty}")

n_biomass = len(merged_df['Biomass'].dropna())
print(f"Number of biomass data associated with image/video/still: {n_biomass}")

n_count = len(merged_df['Count'].dropna())
print(f"Number of count data associated with image/video/still: {n_count}")

n_bysize = len(merged_df['10mm'].dropna())
print(f"Number of sorted mussels size data associated with image/video/still: {n_bysize}")

In [ ]:
simplified_df.columns

In [ ]:
n_alive = len(simplified_df['Live Coverage'].dropna())
print(f"Number of % live coverage data with at least one image: {n_alive}")

n_empty = len(simplified_df['Empty Coverage'].dropna())
print(f"Number of % empty coverage data with at least one image: {n_empty}")

n_biomass = len(simplified_df['Biomass'].dropna())
print(f"Number of biomass data with at least one image: {n_biomass}")

n_count = len(simplified_df['Count'].dropna())
print(f"Number of count data with at least one image: {n_count}")

n_bysize = len(simplified_df['10mm'].dropna())
print(f"Number of sorted mussels size data with at least one image: {n_bysize}")